# Data Loading and Cohort Construction  
**Hospital Length of Stay (LOS) Project — MIMIC-III**

This notebook builds the modeling dataset from raw MIMIC-III tables.

It performs the following steps:
- loads core clinical tables (patients, admissions, ICU stays, diagnoses, prescriptions, services)  
- constructs an ICU-level cohort  
- computes length of stay (LOS) targets  
- engineers early clinical burden features  
- outputs a clean, analysis-ready table

The resulting dataset is used in:
- exploratory data analysis  
- regression modeling (continuous LOS)  
- classification modeling (LOS categories)  
- explainability experiments


## 1. Environment Setup

This section loads the required Python libraries and defines basic configuration settings.  
All transformations and joins are designed to be reproducible.

In [ ]:
from pathlib import Path
import pandas as pd
from datetime import datetime


## 2. File Paths and Project Structure

This section defines file paths for the raw MIMIC-III tables and output directories.

The goal is to keep all I/O logic centralized and make the pipeline easy to rerun on a different machine or folder structure.


In [ ]:
# -----------------------------
# Paths
# -----------------------------
RAW_DIR = Path(r"..\data\raw")          
OUT_DIR = Path(r"..\data\processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)


## 3. Helper Functions

This section defines small utility functions used throughout the notebook.

They handle:
- safe CSV loading  
- datetime parsing  
- column standardization  
- reusable transformations applied across multiple tables


In [95]:
# -----------------------------
# Helpers
# -----------------------------
def read_csv(name, usecols=None, parse_dates=None, low_memory=False):
    path = RAW_DIR / name
    return pd.read_csv(
        path,
        usecols=usecols,
        parse_dates=parse_dates,
        low_memory=low_memory
    )

def to_datetime(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

## 4. Loading Core MIMIC-III Tables

Here we load the minimal set of core tables required for the project:

- `PATIENTS`  
- `ADMISSIONS`  
- `ICUSTAYS`  

Only selected columns are loaded to:
- reduce memory usage  
- avoid unnecessary joins  
- focus on variables relevant for LOS modeling

In [96]:
# -----------------------------
# Load core tables (minimal columns)
# -----------------------------
patients_cols = ["subject_id", "gender", "dob", "dod"]
admissions_cols = [
    "hadm_id", "subject_id", "admittime", "dischtime",
    "admission_type", "admission_location", "insurance","ethnicity"
]
icustays_cols = [
    "icustay_id", "hadm_id", "subject_id",
    "intime", "outtime", "first_careunit", "last_careunit"
]

In [97]:
patients = read_csv("PATIENTS.csv", usecols=patients_cols)
admissions = read_csv("ADMISSIONS.csv", usecols=admissions_cols)
icustays = read_csv("ICUSTAYS.csv", usecols=icustays_cols)

## 5. Timestamp Parsing

All timestamp columns are converted to proper datetime objects.

This is required for:
- computing age at admission  
- computing ICU length of stay  
- filtering invalid or inconsistent records

In [98]:
patients = to_datetime(patients, ["dob", "dod"])
admissions = to_datetime(admissions, ["admittime", "dischtime"])
icustays = to_datetime(icustays, ["intime", "outtime"])

## 6. Basic Data Quality Checks

This step prints basic diagnostics to verify that:

- tables were loaded correctly  
- key identifiers exist  
- timestamps are present and well-formed  
- row counts are in expected ranges

These checks are lightweight sanity tests before constructing the cohort.


In [99]:
# -----------------------------
# Basic QC prints
# -----------------------------
print("PATIENTS:", patients.shape, "unique subject_id:", patients["subject_id"].nunique())
print("ADMISSIONS:", admissions.shape, "unique hadm_id:", admissions["hadm_id"].nunique())
print("ICUSTAYS:", icustays.shape, "unique icustay_id:", icustays["icustay_id"].nunique())

PATIENTS: (100, 4) unique subject_id: 100
ADMISSIONS: (129, 8) unique hadm_id: 129
ICUSTAYS: (136, 7) unique icustay_id: 136


## 7. ICU-Level Cohort Construction

In this step, the unit of analysis is defined as an **ICU stay**.

Tables are merged to form a single cohort table where each row corresponds to:
- one ICU admission  
- one hospital admission  
- one patient

This structure allows LOS to be modeled at the ICU stay level rather than at the patient or hospital-visit level.


In [100]:
# -----------------------------
# Build cohort (ICU stays as unit of analysis)
# -----------------------------
cohort = (
    icustays
    .merge(admissions, on=["hadm_id", "subject_id"], how="left", validate="m:1")
    .merge(patients, on="subject_id", how="left", validate="m:1")
)


In [101]:
# Drop rows with missing ICU timestamps (cannot compute LOS)
cohort = cohort.dropna(subset=["intime", "outtime"]).copy()

## 8. Length of Stay (LOS) Computation

Length of stay is computed at the ICU level.

We derive:
- `los_hours`: total ICU stay duration in hours  
- `los_days`: total ICU stay duration in days  

These values serve as:
- the continuous target for regression models  
- the base variable for LOS category construction


In [102]:
# Target: LOS
cohort["los_hours"] = (cohort["outtime"] - cohort["intime"]).dt.total_seconds() / 3600.0
cohort["los_days"] = cohort["los_hours"] / 24.0

In [103]:
# Remove non-positive LOS (data issues)
cohort = cohort[cohort["los_hours"] > 0].copy()

## 9. Safe Age Computation

Patient age at ICU admission is computed using date of birth and ICU entry time.

A safe function is used to:
- handle missing or implausible dates  
- cap extreme ages  
- avoid negative values caused by timestamp inconsistencies


In [104]:
def compute_age_safe(row):
    dob = row["dob"]
    intime = row["intime"]

    if pd.isna(dob) or pd.isna(intime):
        return None

    # convert to native python datetime (avoids pandas overflow)
    dob = dob.to_pydatetime()
    intime = intime.to_pydatetime()

    age = (intime - dob).days / 365.25
    return age

cohort["age"] = cohort.apply(compute_age_safe, axis=1)

# clip unrealistic ages due to MIMIC de-identification
cohort["age"] = cohort["age"].clip(lower=0, upper=120)


## 10. Base Feature Selection

At this stage, the cohort table is reduced to a core set of variables:

- demographic information  
- admission metadata  
- ICU timestamps  
- LOS targets  

Columns that are not needed for modeling or that could cause leakage are removed.


In [ ]:
# -----------------------------
# Keep only what we need (for now)
# -----------------------------
keep_cols = [
    "subject_id", "hadm_id", "icustay_id",
    "intime", "outtime",
    "los_hours", "los_days",
    "age", "gender",
    "admission_type", "admission_location", "insurance",
    "first_careunit", "last_careunit"
]
cohort = cohort[keep_cols].copy()

print("COHORT:", cohort.shape)
print(cohort[["los_days", "age"]].describe())

COHORT: (136, 15)
         los_days         age
count  136.000000  136.000000
mean     4.452461   72.812014
std      6.196832   19.842476
min      0.105926   17.190965
25%      1.233504   63.833676
50%      2.111447   74.672142
75%      4.329063   83.197810
max     35.406516  120.000000


## 11. Final Dataset Export

The final modeling dataset is saved to disk.

This dataset will be used in:
- exploratory data analysis  
- regression modeling  
- classification modeling  
- explainability experiments  

All downstream notebooks depend on this output.


In [111]:
# -----------------------------
# Save
# -----------------------------
out_path = OUT_DIR / "cohort.parquet"
cohort.to_csv(OUT_DIR / "cohort.csv", index=False)
print("Saved:", out_path)

Saved: ..\data\processed\cohort.parquet


In [112]:
cohort[["los_days", "n_diagnoses", "n_unique_drugs_48h", "n_services"]].describe()

,los_days,n_diagnoses,n_unique_drugs_48h,n_services
count,136.000000,136.000000,136.000000,136.000000
mean,4.452461,13.948529,15.544118,1.205882
std,6.196832,6.454766,10.364375,0.405840
min,0.105926,3.000000,0.000000,1.000000
25%,1.233504,9.000000,8.750000,1.000000
50%,2.111447,13.000000,14.000000,1.000000
75%,4.329063,18.000000,21.250000,1.000000
max,35.406516,37.000000,45.000000,2.000000
